## Opis problemu

Znajdź dowolny zbiór danych (dozwolone języki: angielski, hiszpański, polski, szwedzki) (poza IMDB oraz zbiorami wykorzystywanymi na zajęciach) do analizy sentymentu.
Zbiór może mieć 2 lub 3 klasy.

Następnie:
1. Oczyść dane i zaprezentuj rozkład klas
2. Zbuduj model analizy sentymenu:
  - z wykorzystaniem sieci rekurencyjnej (LSTM/GRU/sieć dwukierunkowa) innej niż podstawowe RNN
  - z wykorzystaniem sieci CNN
  - z podstawiemiem pre-trained word embeddingów
  - z fine-tuningiem modelu języka (poza podstawowym BERTem)

3. Stwórz funkcję, która będzie korzystała z wytrenowanego modelu i zwracała wynik dla przekazanego pojedynczego zdania (zdań) w postaci komunikatu informującego użytkownika, czy tekst jest nacechowany negatywnie, pozytywnie (czy neutralnie w przypadku 3 klas).

4. Gotowe rozwiązanie zamieść na GitHubie z README. W README zawrzyj: informacje o danych - ich pochodzenie, oraz opis wybranego modelu i instrukcje korzystania z plików.
5. W assigmnencie w Teamsach wrzuć link do repo z rozwiązaniem. W przypadku prywatnego repo upewnij się, że będzie ono widoczne dla `dwnuk@pjwstk.edu.pl`.

**TERMIN**: jak w Teamsach

In [46]:

import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from collections import Counter





In [47]:


# todo: opisać zbió® danych

###############
# Load data
###############
file_path = 'data.csv'
data = pd.read_csv(file_path).head(5000)
data = data.drop(columns=['index'])
data['tweets'] = data['tweets'].str.replace('[^a-zA-Z\s]', '').str.lower()

###############
# Prepare datasets
###############
X_train, X_test, y_train, y_test = train_test_split(data['tweets'], data['labels'], test_size=0.2, random_state=42)
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)





In [48]:


###############
# Encoding lavels
###############
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)




In [49]:

###############
# Making tokens
###############
def tokenize(text):
    return text.split()

word_counts = Counter()
for text in X_train:
    word_counts.update(tokenize(text))
vocab = {word: i+1 for i, word in enumerate(word_counts)} # +1 dla paddingu
vocab['<pad>'] = 0



In [50]:


# creating datasets 

class CustomDataset(Dataset):
    def __init__(self, texts, labels, vocab):
        self.texts = texts
        self.labels = labels
        self.vocab = vocab

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        numericalized_text = [self.vocab.get(word, 0) for word in tokenize(text)]  # 0 dla nieznanych slow
        return torch.tensor(numericalized_text, dtype=torch.long), label

def collate_batch(batch):
    text_list, labels = zip(*batch)
    text_tensor = pad_sequence([text for text, _ in batch], batch_first=True, padding_value=0)
    labels_tensor = torch.tensor(labels, dtype=torch.long)
    return text_tensor, labels_tensor

batch_size = 256
train_dataset = CustomDataset(X_train, y_train_encoded, vocab)
test_dataset = CustomDataset(X_test, y_test_encoded, vocab)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_batch)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_batch)





In [51]:

import torch.nn as nn
import torch.optim as optim


# model LSTM
class SentimentAnalysisLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(SentimentAnalysisLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(0.5)

    def forward(self, text):
        embedded = self.embedding(text)
        output, (hidden, cell) = self.lstm(embedded)
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        hidden = self.dropout(hidden)
        return self.fc(hidden)

vocab_size = len(vocab)
embedding_dim = 128
hidden_dim = 256
output_dim = len(le.classes_)

model = SentimentAnalysisLSTM(vocab_size, embedding_dim, hidden_dim, output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.002)





In [72]:

import time
from sklearn.metrics import accuracy_score


def train(model, iterator, optimizer, criterion, device=None):
    model.train()
    for text, labels in iterator:
        optimizer.zero_grad()
        predictions = model(text)
        loss = criterion(predictions, labels)
        loss.backward()
        optimizer.step()

def evaluate(model, iterator, criterion=None, device=None):
    model.eval()
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for text, labels in iterator:
            predictions = model(text)
            all_predictions.extend(predictions.argmax(dim=1).tolist())
            all_labels.extend(labels.tolist())

    accuracy = accuracy_score(all_labels, all_predictions)
    return accuracy

In [71]:


# Training loop
N_EPOCHS = 10
for epoch in range(N_EPOCHS):
    s_t = time.time()
    # print("E", epoch)
    train(model, train_loader, optimizer, criterion)
    # print(1)
    # train_accuracy = evaluate(model, train_loader, criterion)
    # print(2)
    test_accuracy = evaluate(model, test_loader)
    print(f"Epoch: {epoch+1} | Dur: {time.time() - s_t:.3f}s")
    # print(f'\tTrain Accuracy: {train_accuracy * 100:.2f}%')
    print(f'\tTest Accuracy: {test_accuracy * 100:.2f}%')



Epoch: 1 | Dur: 1.566s
	Test Accuracy: 49.70%
Epoch: 2 | Dur: 1.492s
	Test Accuracy: 53.50%
Epoch: 3 | Dur: 1.462s
	Test Accuracy: 58.00%
Epoch: 4 | Dur: 1.488s
	Test Accuracy: 59.30%
Epoch: 5 | Dur: 1.498s
	Test Accuracy: 62.60%
Epoch: 6 | Dur: 1.919s
	Test Accuracy: 61.20%


KeyboardInterrupt: 

In [53]:


torch.save(model.state_dict(), 'models/lstm.pth')
# Recreate the model (ensure the class definition is available)
model = SentimentAnalysisLSTM(vocab_size, embedding_dim, hidden_dim, output_dim)

# Load the saved state dictionary
model.load_state_dict(torch.load('models/lstm.pth'))

# Don't forget to set the model to evaluation mode
model.eval()




SentimentAnalysisLSTM(
  (embedding): Embedding(17372, 128, padding_idx=0)
  (lstm): LSTM(128, 256, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=512, out_features=3, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [77]:


import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from sklearn.metrics import accuracy_score

device = torch.device('cpu')

class TextCNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout):
        super(TextCNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        self.convs = nn.ModuleList([
            nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(fs, embedding_dim))
            for fs in filter_sizes
        ])

        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        
        
    def forward(self, x):
        x = self.embedding(x)  # [batch size, sent len, emb dim]
        x = x.unsqueeze(1)  # [batch size, 1, sent len, emb dim]
        x = [torch.relu(conv(x)).squeeze(3) for conv in self.convs]
        x = [torch.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in x]
        x = torch.cat(x, dim=1)
        x = self.dropout(x)
        return self.fc(x)

vocab_size = len(vocab)
embedding_dim = 100
n_filters = 100
filter_sizes = [2, 3, 4]
output_dim = len(le.classes_)
dropout = 0.5

model = TextCNN(vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout).to(device)


optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

# Training loop
N_EPOCHS = 100
for epoch in range(N_EPOCHS):
    s_t = time.time()
    train(model, train_loader, optimizer, criterion, device)
    # train_accuracy = evaluate(model, train_loader, criterion, device)
    test_accuracy = evaluate(model, test_loader, criterion, device)
    print(f"Epoch: {epoch+1} | Dur: {time.time() - s_t:.3f}s")
    # print(f'\tTrain Accuracy: {train_accuracy * 100:.2f}%')
    print(f'\tTest Accuracy: {test_accuracy * 100:.2f}%')




Epoch: 1 | Dur: 1.653s
	Test Accuracy: 48.90%
Epoch: 2 | Dur: 1.470s
	Test Accuracy: 57.90%
Epoch: 3 | Dur: 1.474s
	Test Accuracy: 54.00%
Epoch: 4 | Dur: 1.433s
	Test Accuracy: 56.90%
Epoch: 5 | Dur: 1.454s
	Test Accuracy: 60.90%
Epoch: 6 | Dur: 1.449s
	Test Accuracy: 62.10%
Epoch: 7 | Dur: 1.456s
	Test Accuracy: 62.70%
Epoch: 8 | Dur: 1.472s
	Test Accuracy: 60.90%
Epoch: 9 | Dur: 1.494s
	Test Accuracy: 61.10%
Epoch: 10 | Dur: 1.488s
	Test Accuracy: 64.30%
Epoch: 11 | Dur: 1.481s
	Test Accuracy: 63.50%
Epoch: 12 | Dur: 1.511s
	Test Accuracy: 63.10%
Epoch: 13 | Dur: 1.488s
	Test Accuracy: 64.60%
Epoch: 14 | Dur: 1.813s
	Test Accuracy: 64.10%
Epoch: 15 | Dur: 1.514s
	Test Accuracy: 64.50%
Epoch: 16 | Dur: 1.607s
	Test Accuracy: 64.30%
Epoch: 17 | Dur: 1.485s
	Test Accuracy: 64.80%
Epoch: 18 | Dur: 1.494s
	Test Accuracy: 63.30%
Epoch: 19 | Dur: 1.653s
	Test Accuracy: 64.20%
Epoch: 20 | Dur: 1.529s
	Test Accuracy: 64.70%
Epoch: 21 | Dur: 1.495s
	Test Accuracy: 62.90%
Epoch: 22 | Dur: 1.488

In [78]:



import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from collections import Counter

# Assuming you have loaded your dataset into a DataFrame `data`

# Load GloVe Embeddings
def load_glove_embeddings(path):
    embeddings_dict = {}
    with open(path, 'r', encoding='utf8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = torch.tensor([float(val) for val in values[1:]], dtype=torch.float)
            embeddings_dict[word] = vector
    return embeddings_dict

glove_path = 'glove.6B.100d.txt'  # Update this path
glove_embeddings = load_glove_embeddings(glove_path)

# Prepare Data
X_train, X_test, y_train, y_test = train_test_split(data['tweets'], data['labels'], test_size=0.2, random_state=42)
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

# Build Vocabulary
vocab = {"<PAD>": 0}
for text in X_train:
    for word in text.split():
        if word not in vocab:
            vocab[word] = len(vocab)

# Prepare Embedding Matrix
embedding_dim = 100  # Dimension of GloVe vectors you are using
embedding_matrix = torch.zeros((len(vocab), embedding_dim))
for word, idx in vocab.items():
    embedding_vector = glove_embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[idx] = embedding_vector
    else:
        embedding_matrix[idx] = torch.randn(embedding_dim)  # Random vector for unknown words

# Model
class SimpleNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim):
        super(SimpleNN, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=True)
        self.fc = nn.Linear(embedding_dim, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)
        pooled = torch.mean(embedded, dim=1)  # Average pooling
        return self.fc(pooled)

# Instantiate model, loss, optimizer
model = SimpleNN(len(vocab), embedding_dim, len(le.classes_))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Data Loaders
# Assuming you have defined CustomDataset and collate_fn appropriately
train_dataset = CustomDataset(X_train, y_train_encoded, vocab)
test_dataset = CustomDataset(X_test, y_test_encoded, vocab)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Training Loop
for epoch in range(10):
    for texts, labels in train_loader:
        # Your training loop: forward pass, compute loss, backward pass
        pass

    # Evaluation on the test set
    # ...

# Save your model if needed
# torch.save(model.state_dict(), 'model.pth')


























FileNotFoundError: [Errno 2] No such file or directory: 'glove.6B.100d.txt'